In [ ]:
!wget https://raw.githubusercontent.com/happylolonly/eventsfree-ml/master/notebooks/tags/data/data.csv

In [ ]:
# !wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/word-vectors-v2/cc.ru.300.bin.gz

In [ ]:
# !wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/word-vectors-v2/cc.ru.300.vec.gz

In [ ]:
# !gunzip cc.ru.300.bin.gz
# !gunzip cc.ru.300.vec.gz

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('./data.csv')
df = df.replace(r'\\n',' ', regex=True) 

df['tags'] = df['tags'].map(eval) 
df['tags'] = df['tags'].map(lambda x: set(x))

df.head()

,tags,text,title
0,{выставка},<p>&#x41D;&#x438;&#x43A;&#x442;&#x43E; &#x43D;...,Письма из Венеции
1,"{фитнес, праздник}",🎄🎄🎄А Вы уже ощущаете праздник и новогоднее нас...,🎄🎄🎄А Вы уже ощущаете праздник и новогоднее на...
2,"{тренировка, фитнес, кроссфит}",🔥🔥🔥PoisonBox дарит возможности!!! 🔥🔥🔥 В эту...,🔥🔥🔥PoisonBox дарит возможности!!! 🔥🔥🔥 В эт...
3,"{тренировка, фитнес, кроссфит }",БЕСПЛАТНАЯ открытая тренировка по CROSSFIT! ...,БЕСПЛАТНАЯ открытая тренировка по CROSSFIT! ...
4,"{вебинар, работа, германия}","🇩🇪 Хотите работать в Германии, но не владеете ...","🇩🇪 Хотите работать в Германии, но не владеете..."


Remove lines where only 1 uniqie tag

In [4]:
from collections import Counter

counter = Counter()
for tags in df['tags'].values:
    for t in tags:
        counter[t] += 1
counter

arr = []
for i in counter:
    if counter[i] > 2:
        arr.append(i);
        
print('total count:', len(counter), '\n >2 tags:', len(arr))

total count: 77 
 >2 tags: 33


In [5]:
tags_white_list = set(arr)
df['tags'] = df['tags'].map(lambda x: set(x).intersection(tags_white_list))
df = df[df.tags.map(len) > 0]

df.shape

(167, 3)

In [6]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
from stop_words import get_stop_words
nltk.download('wordnet')
stemmer = SnowballStemmer('russian')

import json
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def standardize_text(df, text_field):
    df[text_field] = df[text_field].apply(strip_tags)
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"[^а-яА-Яa-zA-Z]", " ")
    df[text_field] = df[text_field].str.replace(r"\s+", " ")
    
    df[text_field] = df[text_field].str.lower()
    
    stop_words = get_stop_words('russian')
    
    def stem (text):
        temp = []
        for word in text.split(" "):
            if len(word) > 2 and word not in stop_words:
                temp.append(stemmer.stem(word))
        return " ".join(temp)
        
            
    df[text_field] = df[text_field].apply(stem)
    
    return df

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/happylol/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
df = standardize_text(df, 'text')
df = standardize_text(df, 'title')

df.head()

,tags,text,title
0,{выставка},никт оста равнодушн венец город тысяч лиц суще...,письм венец
1,"{фитнес, праздник}",ощуща праздник новогодн настроен декабр фитнес...,ощуща праздник новогодн настроен декабр
2,"{тренировка, фитнес}",poisonbox дар возможн суббот декабр состо откр...,poisonbox дар возможн суббот декабр состо откр...
3,"{тренировка, фитнес}",бесплатн открыт тренировк crossf приход проб д...,бесплатн открыт тренировк crossf приход проб д...
6,"{тренировка, фитнес, trx}",вниман открыт занят trx super набор дневн груп...,вниман открыт занят trx super набор дневн групп


In [8]:
rows = []

for row in df.to_dict(orient='record'):
    for tag in row['tags']:
        row_copy = row.copy()
        row_copy['tag'] = tag
        
        rows.append(row_copy)
        
df = pd.DataFrame(rows)

df['fulltext'] = df[['text', 'title']].apply(lambda x: ' '.join(x), axis=1)

print(df.shape)

(311, 5)


In [11]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3, stratify=df['tag'].values)

print(np.unique(train['tag'].values, return_counts=True))
print(np.unique(test['tag'].values, return_counts=True))

(array(['DJ', 'english', 'it', 'trx', 'вечеринка', 'выставка', 'гитара',
       'дети', 'джаз', 'дизайн', 'инвестиции', 'кино', 'конкурс',
       'концерт', 'лекция', 'маркет', 'митап', 'музыка', 'нетворкинг',
       'обучение', 'праздник', 'программирование', 'семинар', 'стартап',
       'танцы', 'театр', 'тенис', 'тренировка', 'турнир', 'фильм',
       'фитнес', 'фотография', 'ярмарка'], dtype=object), array([ 6,  3,  6,  2, 18,  5,  3, 12,  3,  3,  3,  3,  2,  2,  6,  3, 19,
       33,  3,  9, 12,  3,  2,  3,  3,  4,  3, 11,  3,  3, 13,  2, 11]))
(array(['DJ', 'english', 'it', 'trx', 'вечеринка', 'выставка', 'гитара',
       'дети', 'джаз', 'дизайн', 'инвестиции', 'кино', 'конкурс',
       'концерт', 'лекция', 'маркет', 'митап', 'музыка', 'нетворкинг',
       'обучение', 'праздник', 'программирование', 'семинар', 'стартап',
       'танцы', 'театр', 'тенис', 'тренировка', 'турнир', 'фильм',
       'фитнес', 'фотография', 'ярмарка'], dtype=object), array([ 2,  2,  3,  1,  8,  2,  2,  

In [55]:
# %%bash

# pip install -U -q git+https://github.com/facebookresearch/fastText.git

import fastText

FIELD = 'fulltext'

In [56]:
def save(df, path):
    with open(path, 'w+') as f:
        for _, row in df.iterrows():
            f.write('__label__{} {}\n'.format(row['tag'], row[FIELD]))

In [57]:
# save(df, '/tmp/train.data')
save(train, '/tmp/train.data')

In [58]:
model = fastText.train_supervised('/tmp/train.data',
                                  pretrainedVectors='/tmp/ml/fasttext/cc.ru.300.vec',
                                  dim=300,
                                  thread=15,
                                  epoch=50,
                                  verbose=8)

In [59]:
labels, probs = model.predict(test[FIELD].tolist(), k=5)
labels = [ll[0].replace('__label__', '') for ll in labels]

In [60]:
from sklearn.metrics import classification_report
print(classification_report(test['tag'].values, labels))

                  precision    recall  f1-score   support

              DJ       0.00      0.00      0.00         2
         english       0.00      0.00      0.00         2
              it       0.00      0.00      0.00         3
             trx       0.00      0.00      0.00         1
       вечеринка       0.14      0.12      0.13         8
        выставка       0.50      0.50      0.50         2
          гитара       0.00      0.00      0.00         2
            дети       0.00      0.00      0.00         5
            джаз       0.50      0.50      0.50         2
          дизайн       0.00      0.00      0.00         1
      инвестиции       0.00      0.00      0.00         1
            кино       0.00      0.00      0.00         1
         конкурс       0.00      0.00      0.00         1
         концерт       0.00      0.00      0.00         1
          лекция       0.00      0.00      0.00         2
          маркет       0.00      0.00      0.00         1
           ми

/Users/happylol/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Production model

In [18]:
%%time
save(df, '/tmp/train.data')
model = fastText.train_supervised('/tmp/train.data',
                                  pretrainedVectors='/tmp/ml/fasttext/cc.ru.300.vec',
                                  dim=300,
                                  thread=15,
                                  epoch=50,
                                  verbose=8)

model.quantize(qnorm=True, cutoff=100000)
model.save_model('tags_model')

In [61]:
labels, probs = model.predict(test[FIELD].tolist(), k=5)

In [62]:
mistakes = 0;
for i, item in enumerate(test['tags']):
    length = len(item)
    
    prediction = map(lambda x: x.replace('__label__', ''), labels[i][0:length:])
    prediction = set(prediction)
    
    if (item != prediction):
        print(item, prediction)
        mistakes += 1
        
print('Accuracy:', (test.shape[0] - mistakes) / test.shape[0])

{'вечеринка', 'музыка'} {'гитара', 'музыка'}
{'музыка', 'вечеринка', 'гитара'} {'вечеринка', 'джаз', 'музыка'}
{'вечеринка', 'музыка', 'конкурс'} {'стартап', 'вечеринка', 'конкурс'}
{'митап', 'it', 'дизайн'} {'программирование', 'митап', 'it'}
{'дети', 'обучение', 'лекция'} {'дети', 'тенис', 'праздник'}
{'дети', 'тренировка'} {'тренировка', 'фитнес'}
{'семинар', 'инвестиции', 'митап'} {'семинар', 'инвестиции', 'trx'}
{'ярмарка', 'фотография', 'вечеринка'} {'обучение', 'фотография', 'вечеринка'}
{'ярмарка'} {'маркет'}
{'фотография', 'выставка'} {'ярмарка', 'выставка'}
{'вечеринка', 'музыка'} {'вечеринка', 'танцы'}
{'семинар', 'обучение'} {'обучение', 'лекция'}
{'DJ', 'музыка'} {'DJ', 'вечеринка'}
{'нетворкинг', 'инвестиции', 'митап'} {'семинар', 'инвестиции', 'митап'}
{'маркет'} {'ярмарка'}
{'фотография', 'выставка'} {'ярмарка', 'выставка'}
{'нетворкинг', 'ярмарка', 'it'} {'стартап', 'нетворкинг', 'митап'}
{'дети', 'театр'} {'дети', 'праздник'}
{'english'} {'обучение'}
{'стартап', 'мита